# Job Application Client Notebook

This notebook provides an interactive interface for running the job application process using the CrewAI framework.

In [1]:
import sys
import os

# Add the parent directory to sys.path to be able to import the crew module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from crewai import Crew
from crew.agents import create_agents
from crew.tasks import create_tasks
from crew.utils import load_config, print_llm_assignments, convert_md_to_pdf, convert_ltx_to_pdf

/home/kedar/miniconda3/envs/rag-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kedar/miniconda3/envs/rag-pipeline/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## Configuration

Set the paths for your resume and configuration file.

In [2]:
resume_path = "../resume.md"
config_path = "config_1.json"

# Load configuration
config = load_config(config_path)

# Check for Serper API key
if 'serper' not in config['api_keys'] or not config['api_keys']['serper']:
    raise ValueError("Serper API key is missing in the config file. Please add it to continue.")

# Print LLM assignments
print_llm_assignments(config)

Set OPENAI_API_KEY environment variable
Set ANTHROPIC_API_KEY environment variable
Set GOOGLE_API_KEY environment variable
Set SERPER_API_KEY environment variable
LLM assignments:
Job Analyzer: openai - gpt-4o-mini
Relevance Selector: openai - gpt-4o-mini
Emphasis Strategist: openai - gpt-4o-mini
Cover Letter Writer: openai - gpt-4o-mini


## Job Information

Enter the job posting URL and optionally, the job description.

In [3]:
job_posting_url = input("Enter the job posting URL: ")
job_description = input("Enter the job description (optional, press Enter to skip): ")

# Add these to the config
config['job_posting_url'] = job_posting_url
config['job_description'] = job_description

## Create Agents and Tasks

In [4]:
# Create agents
job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer = create_agents(resume_path, config)

# Create tasks
job_analysis_task, relevance_task, emphasis_task, cover_letter_task = create_tasks(
    job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer
)

## Create and Run the Crew

In [5]:
def run_job_application_process(resume_path, config):
    # Create and run the crew
    job_application_crew = Crew(
        agents=[job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer],
        tasks=[job_analysis_task, relevance_task, emphasis_task, cover_letter_task],
        verbose=True
    )
    # Prepare inputs for the crew
    job_application_inputs = {
        'job_posting_url': config['job_posting_url'],
        'super_resume_path': resume_path,
        'job_description': config['job_description']
    }

    # Run the crew
    result = job_application_crew.kickoff(inputs=job_application_inputs)
    return result

print("Starting job application process...")
result = run_job_application_process(resume_path, config)

print("\nJob Application Process Completed")
print("\nResults:")
print(result)




Starting job application process...


2024-10-04 17:43:26,820 - 139901748110912 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.8s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f3d697d1c00>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))


# Agent: Tech Job Researcher and Analyzer
## Task: Analyze the job posting URL provided (https://jobs.lever.co/valence/03c74aad-9aa4-4d7c-823c-2225688ba510) to extract key skills, experiences, and qualifications required. If you cannot scrape the job description or no relevant information is available from the URL, use (Machine Learning Engineer New York, New York / Boston, Massachusetts / Philadelphia, Pennsylvania / Washington, DC Engineering / Full-Time (Remote-Friendly) / Remote               Valence has built the first-ever, first-to-market AI coaching platform for enterprise, offering personalized, expert, and human-like guidance and support to any leader or employee. At Valence, we're not just talking about the future of work – we're actively shaping it.  From your first interaction with us, you'll notice we're different. We're not just offering a job; we're inviting you to help transform the future of work by bringing groundbreaking research into real-world applications that wi

2024-10-04 17:43:27,771 - 139901748110912 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.4s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f3d69721cf0>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:43:28,286 - 139901748110912 - _common.py-_common:105 - INFO: Backing off send_request(...) for 3.9s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f3d69720a30>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))




# Agent: Tech Job Researcher and Analyzer
## Thought: I need to access the job posting URL to extract the job description and analyze key requirements.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://jobs.lever.co/valence/03c74aad-9aa4-4d7c-823c-2225688ba510\"}"
## Tool Output: 
Valence - Machine Learning EngineerMachine Learning EngineerNew York, New York / Boston, Massachusetts / Philadelphia, Pennsylvania / Washington, DCEngineering /Full-Time (Remote-Friendly) /RemoteApply for this jobValence has built the first-ever, first-to-market AI coaching platform for enterprise, offering personalized, expert, and human-like guidance and support to any leader or employee. At Valence, we're not just talking about the future of work – we're actively shaping it.From your first interaction with us, you'll notice we're different. We're not just offering a job; we're inviting you to help transform the future of work by bringing groundbreaking research into real-w

...



# Agent: Relevance Selector
## Final Answer: 
```markdown
# K G

## EDUCATION

**Master of Science, Robotics And Autonomous Systems (Artificial Intelligence)** | August 2023 - May 2025  
Arizona State University  
- **Courses**: Frontiers in GenAI, Operational Deep Learning, ML accelerator design  

**Bachelor of Engineering, Computer Engineering** | August 2015 - May 2019  
University of Mumbai  
- **Courses**: Machine Learning, Artificial Intelligence  

---

## SKILLS

- **Languages & Technologies**: Python, C++, Docker, Git, Jira, GCP, AWS  
- **ML Frameworks**: PyTorch, TensorFlow, Scikit-learn, ONNX, TensorRT, OpenVino  
- **LLM Frameworks**: OpenAI, LangGraph, Langchain, LlamaIndex, Ollama, LangSmith  
- **Embedded Systems**: Nvidia Jetson, Ambarella CV22, Raspberry Pi  

---

## EXPERIENCE

### AI Research Intern, [RagaAI](https://www.raga.ai/) - Fremont, CA | June 2024 - Present  
- Developed an observability tool, **RagaAI Catalyst**, providing trace recording for RAG appli

2024-10-04 17:44:12,875 - 139901748110912 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.5s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f3d684ba8f0>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))




# Agent: Latex Resume Strategist
## Thought: Thought: I have successfully read the content of the resume. Now I will extract the necessary information to format it into the provided LaTeX template.
## Using tool: Search a MDX's content
## Tool Input: 
"{\"search_query\": \"K G\"}"
## Tool Output: 
Relevant Content:
- Reduced deep learning model size by 80% via model pruning and quantization, maintaining performance and enhancing data processing efficiency, leading to 30% deployment cost savings
- Integrated neuro-symbolic approaches with Explainable AI (XAI) to enhance neural network performance, resulting in a 30% boost in object detection recall for geospatial imaging securing an Air Force/Space Force STTR Phase I contract
- Developed a stress-testing framework for object detectors which performs targeted adversarial patch attacks, camouflage and contrast changes to simulate real life situations and test the model's robustness
- Engineered an object tracking algorithm featuring mot

2024-10-04 17:44:13,451 - 139901748110912 - _common.py-_common:105 - INFO: Backing off send_request(...) for 1.0s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f3d682fd000>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:44:14,556 - 139901748110912 - _common.py-_common:105 - INFO: Backing off send_request(...) for 2.4s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f3d682fca30>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:44:17,069 - 139901748110912 - _common.py-_common:120 - ERROR: Giving up send_request(...) after 4 tries 



# Agent: Latex Resume Strategist
## Final Answer: 
```latex
\documentclass[resume]
\usepackage[implicit=false][hyperref]
\usepackage[enumitem]
\setlist[topsep=-3pt, itemsep=-3pt]
\usepackage[left=0.45in,top=0.4in,right=0.45in,bottom=0.4in]geometry
\newcommand\tab[1]\hspace.2667\textwidth\rlap#1
\newcommand\MYhref[3][blue]\href#2\color#1#3
\newcommand\itab[1]\hspace0em\rlap#1

\name\Large K G
\address\href[mailto:kg@example.com]{kg@example.com} \\ \href[https://www.linkedin.com/in/k-g]{www.linkedin.com/in/k-g} \\ 
\href[tel:1234567890]{1234567890} \\ New York, NY

\begin{document}

%----------------------------------------------------------------------------------------
%	EDUCATION SECTION
%----------------------------------------------------------------------------------------
\begin{rSection}Education

\bf Master of Science, Robotics And Autonomous Systems (Artificial Intelligence) $|$ Arizona State University \hfill August 2023 - May 2025 \\
\bf Courses: Frontiers in GenAI, Operati

## Review Output Files

The process should have generated several output files. Let's review them:

In [6]:
output_files = ['resume.md', 'latex_resume.md', 'cover_letter.md']

for file in output_files:
    print(f"\nContents of {file}:")
    print("-" * 40)
    try:
        with open(file, 'r') as f:
            print(f.read())
    except FileNotFoundError:
        print(f"File {file} not found. It may not have been generated.")


Contents of resume.md:
----------------------------------------
```markdown
# K G

## EDUCATION

**Master of Science, Robotics And Autonomous Systems (Artificial Intelligence)** | August 2023 - May 2025  
Arizona State University  
- **Courses**: Frontiers in GenAI, Operational Deep Learning, ML accelerator design  

**Bachelor of Engineering, Computer Engineering** | August 2015 - May 2019  
University of Mumbai  
- **Courses**: Machine Learning, Artificial Intelligence  

---

## SKILLS

- **Languages & Technologies**: Python, C++, Docker, Git, Jira, GCP, AWS  
- **ML Frameworks**: PyTorch, TensorFlow, Scikit-learn, ONNX, TensorRT, OpenVino  
- **LLM Frameworks**: OpenAI, LangGraph, Langchain, LlamaIndex, Ollama, LangSmith  
- **Embedded Systems**: Nvidia Jetson, Ambarella CV22, Raspberry Pi  

---

## EXPERIENCE

### AI Research Intern, [RagaAI](https://www.raga.ai/) - Fremont, CA | June 2024 - Present  
- Developed an observability tool, **RagaAI Catalyst**, providing trace record

In [ ]:
convert_ltx_to_pdf('latex_resume.md', os.path.join(os.getcwd(), "crew", 'resume.cls'), output_pdf_name="resume.pdf")
convert_md_to_pdf(['cover_letter.md'])

## Next Steps

1. Review the generated files and make any necessary adjustments.
2. Use the focused resume and tailored cover letter for your job application.
3. Consider the emphasis strategy when formatting your resume.

Good luck with your job application!